<a href="https://colab.research.google.com/github/Cullen-hub/ML_and_AI_Course/blob/main/Rock_Paper_Sissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!wget https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/RPS.py
#!wget https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/main.py

In [ ]:
#!wget https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/RPS_game.py
#!wget https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/test_module.py


In [ ]:
#!wget -O RPS.py https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/RPS.py
#!wget -O main.py https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/main.py
#!wget -O RPS_game.py https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/RPS_game.py
#!wget -O test_module.py https://raw.githubusercontent.com/Cullen-hub/Rock_P_S_project/main/test_module.py


In [ ]:
#imports
from collections import defaultdict
from collections import Counter
import unittest
from unittest import main
import math
import random



In [ ]:
#RPS.py
# The example function below keeps track of the opponent's history and plays whatever the opponent played two plays ago. It is not a very good player so you will need to change the code to pass the challenge.

def n_gram(n, opponent_history, actions):
        dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        Current_pattern = tuple(opponent_history[-n:])

        for i in range(len(opponent_history)-n):
            pattern = tuple(opponent_history[i:i+n])
            players_next_play = opponent_history[i+n]
            dictionary_Of_Events[pattern][players_next_play] += 1

        for i in actions:
            if dictionary_Of_Events[Current_pattern].get(i) == None:
                dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(dictionary_Of_Events[Current_pattern].values())

        if total_sum == 0:
          guess = None
          weighted_confidence_level = 0
        else:
          guess = max(dictionary_Of_Events[Current_pattern], key = dictionary_Of_Events[Current_pattern].get)
          Max_value = max(dictionary_Of_Events[Current_pattern].values())
          sorted_dict = sorted(dictionary_Of_Events[Current_pattern].values(), reverse = True)
          second_highest_value = sorted_dict[1]
          confidence_level = (Max_value - second_highest_value)/total_sum
          weight =(n / 20) * (total_sum / (len(opponent_history)+1))
          weighted_confidence_level = confidence_level * weight

        return guess,  weighted_confidence_level



def player(prev_play, opponent_history=[]):
    global My_history
    try:
      My_history = My_history
    except:
      My_history = []

    actions = ['R', 'P', 'S']

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}

        for i in range(1,min(round(m),20)):
            guess,weighted_confidence_level = n_gram(i, opponent_history, actions)
            if guess != None and guess != '':
                predicted_optimal_guesses[guess] +=  weighted_confidence_level

        My_last_previous_play = My_history[-1]
        High_biases = [0.4,0.5,0.6,0.7]
        Mid_biases = [0.1,0.2,0.3]
        Low_biases = [0.01,0.05,0.08,0.09]
        if len(My_history) >=3:
          if My_history[-1] == My_history[-3] == My_history[-2]:
            predicted_optimal_guesses[My_last_previous_play] -= random.choice(Mid_biases)
          elif  My_history[-1] == My_history[-2]:
            predicted_optimal_guesses[My_last_previous_play] -= random.choice(Mid_biases)
          else:
            predicted_optimal_guesses[My_last_previous_play] -= random.choice(High_biases)

          My_last_previous_play_index = actions.index(My_last_previous_play)
          if My_last_previous_play_index == 2:
            predicted_optimal_guesses[actions[0]] += random.choice(Mid_biases)
          else:
             predicted_optimal_guesses[actions[My_last_previous_play_index + 1]] += random.choice(Mid_biases)


        Possible_opponent_guess, Their_confidence_level = n_gram(2,My_history, actions)
        if Possible_opponent_guess != None and Their_confidence_level > 0.1:
          opponents_possible_actions ={'R': 'P', 'P': 'S', 'S': 'R'}[Possible_opponent_guess]
          Best_counter = {'R': 'P', 'P': 'S', 'S': 'R'}[opponents_possible_actions]
          predicted_optimal_guesses[Best_counter] += 0.4
          predicted_optimal_guesses[opponents_possible_actions] -= 0.3

        for i in actions:
          if predicted_optimal_guesses[i] < 0:
            predicted_optimal_guesses[i] = 0
          #print (predicted_optimal_guesses[i])

        if predicted_optimal_guesses['R'] == predicted_optimal_guesses['S'] == predicted_optimal_guesses['P'] :
            counter_play = random.choice(actions)

        else:
            Optimal_guess =  max(predicted_optimal_guesses, key = predicted_optimal_guesses.get)
            Optimal_Value =  predicted_optimal_guesses[Optimal_guess]
            potential_draw_guess = max(sorted(predicted_optimal_guesses.keys(), reverse = True), key = predicted_optimal_guesses.get)
            if Optimal_Value < 0.4:
              counter_play = random.choice(actions)
            else:
              Overall_Optimal_guess = random.choice([Optimal_guess,potential_draw_guess])
              optimal_guess_index = actions.index(Overall_Optimal_guess)
              if optimal_guess_index == 2:
                  counter_play = actions[0]
              else:
                  counter_play = actions[optimal_guess_index + 1]


    else:
        counter_play = random.choice(actions)

    #if len(My_history) < 20:
      #decoy_pattern = ['P', 'R', 'S']
      #counter_play = decoy_pattern[len(My_history)%3]

    My_history.append(counter_play)

    opponent_history.append(prev_play)
    return counter_play

In [ ]:
#AI help

My_history = []
opponent_history = []

def entropy(prob_dist, actions):
    ent = 0
    for p in prob_dist.values():
        if p > 0:
            ent -= p * math.log2(p)
    max_ent = math.log2(len(actions))
    return 1 - ent / max_ent if max_ent else 0

def n_gram(n, source_history, actions):
    if len(source_history) < n:
        return None, {}, 0
    pattern = tuple(source_history[-n:])
    counts = defaultdict(lambda: defaultdict(int))

    for i in range(len(source_history) - n):
        key = tuple(source_history[i:i+n])
        next_play = source_history[i+n]
        counts[key][next_play] += 1

    current = counts.get(pattern, {})
    total = sum(current.values())
    if total == 0:
        return None, {}, 0

    prob_dist = {a: current.get(a, 0)/total for a in actions}
    guess = max(prob_dist, key=prob_dist.get)
    conf = entropy(prob_dist, actions)

    weight = (n / 20) * (total / (len(source_history) + 1))
    weighted_conf = conf * weight
    return guess, prob_dist, weighted_conf

def markov_prediction(opponent_history):
    if len(opponent_history) < 2:
        return None, 0
    transitions = {'R': {'R':0, 'P':0, 'S':0},
                   'P': {'R':0, 'P':0, 'S':0},
                   'S': {'R':0, 'P':0, 'S':0}}
    for i in range(1, len(opponent_history)):
        prev = opponent_history[i-1]
        curr = opponent_history[i]
        transitions[prev][curr] += 1

    last = opponent_history[-1]
    next_probs = transitions[last]
    total = sum(next_probs.values())
    if total == 0:
        return None, 0
    guess = max(next_probs, key=next_probs.get)
    confidence = next_probs[guess] / total
    return guess, confidence

def player(prev_play, opponent_history_local=[]):
    global My_history, opponent_history

    actions = ['R', 'P', 'S']
    counter = {'R': 'P', 'P': 'S', 'S': 'R'}

    if prev_play == "":
        My_history.clear()
        opponent_history.clear()
        return random.choice(actions)

    opponent_history.append(prev_play)

    predicted_optimal = {'R': 0, 'P': 0, 'S': 0}

    n_max = min(len(opponent_history) // 2, 20)
    for n in range(1, n_max + 1):
        # Normal opponent-based n-gram
        guess, _, weight = n_gram(n, opponent_history, actions)
        if guess:
            boost = 1.2 if n == 2 else 1.0
            predicted_optimal[guess] += weight * boost

        # Reverse modeling: opponent trying to counter you
        reverse_guess, _, reverse_weight = n_gram(n, My_history, actions)
        if reverse_guess:
            opp_guess = counter[reverse_guess]  # they might counter us
            predicted_optimal[counter[opp_guess]] += reverse_weight * 0.6

    # Markov prediction as backup
    markov_guess, markov_conf = markov_prediction(opponent_history)
    if markov_guess and markov_conf > 0.3:
        predicted_optimal[markov_guess] += markov_conf * 0.5

    # Bias adjustment for own repetition
    if len(My_history) >= 3:
        last = My_history[-1]
        if My_history[-1] == My_history[-2] == My_history[-3]:
            predicted_optimal[last] -= 0.3
        elif My_history[-1] == My_history[-2]:
            predicted_optimal[last] -= 0.15
        predicted_optimal[counter[last]] += 0.1

    # Clip negatives
    for a in actions:
        predicted_optimal[a] = max(predicted_optimal[a], 0)
        my_freq = {a: My_history.count(a) for a in actions}
        most_used = max(my_freq, key=my_freq.get)
        predicted_optimal[most_used] *= 0.85

    if all(v == 0 for v in predicted_optimal.values()) or len(opponent_history) < 25:
        counter_play = random.choice(actions)
    else:
        best_guess = max(predicted_optimal, key=predicted_optimal.get)
        counter_play = counter[best_guess]

    if random.random() < 0.1:
        counter_play = random.choice([a for a in actions if a != counter_play])

    My_history.append(counter_play)
    return counter_play

In [ ]:
#TWICK
My_history = []
opponent_history = []
def entropy(Probibility_Of_Events, actions):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          Max_entropy = math.log2(len(actions))
          if Max_entropy == 0:
            confidence_level = 0
          else:
            confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def markov_ngram(opponent_history, n=2):
    if len(opponent_history) < n + 1:
        return None, 0
    transitions = defaultdict(lambda: defaultdict(int))
    for i in range(n, len(opponent_history)):
        prev = tuple(opponent_history[i - n:i])
        next_move = opponent_history[i]
        transitions[prev][next_move] += 1

    last = tuple(opponent_history[-n:])
    if last not in transitions:
        return None, 0

    next_probs = transitions[last]
    total = sum(next_probs.values())
    guess = max(next_probs, key=next_probs.get)
    confidence = next_probs[guess] / total
    return guess, confidence

def n_gram(n, opponent_history, My_history, actions, Type):
        counter = {'R':'P', 'P': 'S','S':'R'}
        dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        if Type == 'Player':
              Current_pattern = tuple(opponent_history[-n:])
              decay_rate = 0.99  # exponential decay
              for i in range(len(opponent_history) - n):
                pattern = tuple(opponent_history[i:i+n])
                players_next_play = opponent_history[i+n]
                weight = decay_rate ** (len(opponent_history) - i)  # more recent = higher weight
                dictionary_Of_Events[pattern][players_next_play] += weight
              history = opponent_history

        elif Type =='Me':
            Current_pattern = tuple(My_history[-n:])
            #for i in range(len(opponent_history)- n):
                #pattern = tuple(My_history[i:i+n])
                #players_next_play = opponent_history[i+n]
                #dictionary_Of_Events[pattern][players_next_play] += 1

            for i in range(len(My_history)- n):
                pattern = tuple(My_history[i:i+n])
                players_next_play = My_history[i+n]
                dictionary_Of_Events[pattern][players_next_play] += 1
            history = My_history

        for i in actions:
            if dictionary_Of_Events[Current_pattern].get(i) == None:
                dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(dictionary_Of_Events[Current_pattern].values())
        Probibility_Of_Events = {}
        if total_sum == 0:
          weight = 0
          guess = None
        else:
          for i in actions:
              Probibility_Of_Events[i] = dictionary_Of_Events[Current_pattern][i]/total_sum

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

          weight = (n / 5) * (total_sum / (len(history) + 1))


        return guess, Probibility_Of_Events, weight


def detect_delayed_counter(My_history, opponent_history):
    max_lag = round(len(My_history)/2)
    if len(My_history) < max_lag + 5:
        return None

    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    best_lag = None
    best_match_score = 0

    for lag in range(1, max_lag + 1):
        match_count = 0
        comparisons = 0
        for i in range(lag, len(My_history)):
            expected = counter[My_history[i - lag]]
            if opponent_history[i] == expected:
                match_count += 1
            comparisons += 1

        if comparisons == 0:
            match_score = 0
        else:
            match_score = match_count / comparisons

        if match_score > best_match_score:
            best_match_score = match_score
            best_lag = lag

    if best_match_score > 0.80:
        return best_lag
    return None



def player(prev_play, opponent_history=[]):

    global My_history

    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    if prev_play == "":
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}


        lag = detect_delayed_counter(My_history, opponent_history)
        if lag != None:
          Opponent_guess = My_history[-lag]
          Opponent_Counter_play = counter[Opponent_guess]
          counter_play = counter[Opponent_Counter_play]
          My_history.append(counter_play)
          return counter_play


        for i in range(1,min(round(m),7)):
            boost = 1.0 + (i / (m + 1))
            boost = min(boost, 1.3)

            guess_Player_history, Probibility_Of_Events_PH, weight_PH = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probibility_Of_Events_MH, weight_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probibility_Of_Events_PH, actions)
            confidence_level_MH = entropy(Probibility_Of_Events_MH, actions)

            if confidence_level_MH < confidence_level_PH:
              confidence_level = confidence_level_PH
              guess = guess_Player_history
              Type = 'Player'
              weighted_confidence_level = weight_PH * confidence_level * boost
            else:
              confidence_level = confidence_level_MH
              Opponent_guessing_me = guess_My_history
              if Opponent_guessing_me == None:
                  guess = None
              else:
                  guess = counter[Opponent_guessing_me]
              Type = 'Me'
              weighted_confidence_level = weight_MH * confidence_level * boost

            if guess != None and guess != '':
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))

        for n in [2, 3, 4,5]:
            markov_guess, markov_conf = markov_ngram(opponent_history, n=n)
            if markov_guess and markov_conf > 0.3:
                predicted_optimal_guesses[markov_guess] += math.log2(1 + markov_conf * n)


        frequency = {}
        frequency = defaultdict(lambda: defaultdict(int))
        for play in actions:
          frequency[play] = My_history.count(play)
        most_frequent = max(frequency, key=frequency.get)
        predicted_optimal_guesses[most_frequent] *= 0.80

        if len(My_history) >=3:
          if My_history[-1] == My_history[-3] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.5
          elif  My_history[-1] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.8
          else:
            predicted_optimal_guesses[counter[My_history[-1]]] += 0.1

        if len(My_history) > 5:
            my_last = My_history[-1]
            beaten = {'R': 'P', 'P': 'S', 'S': 'R'}
            loss_count = sum(1 for i in range(-5, 0) if opponent_history[i] == beaten[My_history[i]])
            if loss_count > 3:
                counter_play = random.choice(actions)

        for play in actions:
          if predicted_optimal_guesses[play] < 0:
            predicted_optimal_guesses[play] = 0

        count = 0
        for values in predicted_optimal_guesses.values():
          if values == 0:
            count += 1
        if count == 3 or len(opponent_history) < 25:
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)


        counter_play = counter[predicted_move]
        remaining_actions = actions
        remaining_actions.remove(counter_play)

        combined_entropy = (confidence_level_MH + confidence_level_PH) / 2
        jitter_threshold = 0.3 * (1 - combined_entropy)
        if random.random() < jitter_threshold:
            counter_play = random.choice([a for a in actions if a != predicted_move])
    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)

    return counter_play

In [ ]:
My_history = []
opponent_history = []
def entropy(Probibility_Of_Events, actions):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          Max_entropy = math.log2(len(actions))
          if Max_entropy == 0:
            confidence_level = 0
          else:
            confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def n_gram(n, opponent_history, My_history, actions, Type):
        dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        if Type == 'Player':
            Current_pattern = tuple(opponent_history[-n:])
            for i in range(len(opponent_history)-n):
                pattern = tuple(opponent_history[i:i+n])
                players_next_play = opponent_history[i+n]
                dictionary_Of_Events[pattern][players_next_play] += 1

        elif Type =='Me':
            if len(My_history) < n or len(opponent_history) <= n:
                return None, {}
            Current_pattern = tuple(My_history[-n:])
            for i in range(1,len(opponent_history)-(n)):
                pattern = tuple(My_history[i:i+n])
                players_next_play = opponent_history[i+n]
                dictionary_Of_Events[pattern][players_next_play] += 1

        for i in actions:
            if dictionary_Of_Events[Current_pattern].get(i) == None:
                dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(dictionary_Of_Events[Current_pattern].values())
        Probibility_Of_Events = {}
        if total_sum == 0:
          guess = None
        else:
          for i in actions:
              Probibility_Of_Events[i] = dictionary_Of_Events[Current_pattern][i]/total_sum

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

        return guess, Probibility_Of_Events


def player(prev_play, opponent_history=[]):

    global My_history

    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    if prev_play == "":
        My_history = []
        opponent_history.clear()
        return random.choice(actions)

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}

        for i in range(min(round(m),50)):
            guess_Player_history, Probibility_Of_Events_PH= n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probibility_Of_Events_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probibility_Of_Events_PH, actions)
            confidence_level_MH = entropy(Probibility_Of_Events_MH, actions)

            if confidence_level_MH < confidence_level_PH:
              confidence_level = confidence_level_PH
              guess = guess_Player_history
              Type = 'Player'
            else:
              confidence_level = confidence_level_MH
              guess = guess_My_history
              Type = 'Me'

            if guess != None and guess != '':

              if confidence_level < 0.4:
                predicted_optimal_guesses[guess] += 0
              else:
                predicted_optimal_guesses[guess] += max(0, (confidence_level - 0.4))**2


        #if predicted_optimal_guesses['R'] == predicted_optimal_guesses['S'] == predicted_optimal_guesses['P'] :
  #          counter_play = random.choice(actions)
#
 #       else:
  #          Optimal_guess =  max(predicted_optimal_guesses, key = predicted_optimal_guesses.get)
   #         potential_draw_guess = max(sorted(predicted_optimal_guesses.keys(), reverse = True), key = predicted_optimal_guesses.get)
    #        Overall_Optimal_guess = random.choice([Optimal_guess,potential_draw_guess])
     #       if Type == 'Me':
      #        counter = {'R':'P', 'P':'S','S':'R'}
       #     else:
        #      counter = {'R': 'R', 'P': 'P', 'S': 'S'}
         #   counter_play = counter[Overall_Optimal_guess]

        if all(v == 0 for v in predicted_optimal_guesses.values()):
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)

        if Type == 'Me':
            counter = {'R':'P', 'P':'S','S':'R'}
        else:
            counter = {'R': 'S', 'P': 'R', 'S': 'P'}
        counter_play = counter[predicted_move]
    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)

    return counter_play

In [ ]:
My_history = []
opponent_history = []
def entropy(Probability_Of_Events, actions):
          entropy = 0
          for probability in Probability_Of_Events:
              if Probability_Of_Events[probability] != 0:
                  entropy = entropy - Probability_Of_Events[probability] * math.log2(Probability_Of_Events[probability])
          Max_entropy = math.log2(len(actions))
          if Max_entropy == 0:
            confidence_level = 0
          else:
            confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def markov_2gram(opponent_history):
    if len(opponent_history) < 3:
        return None, 0
    transitions = defaultdict(lambda: defaultdict(int))
    for i in range(2, len(opponent_history)):
        prev2 = tuple(opponent_history[i-2:i])
        next_move = opponent_history[i]
        transitions[prev2][next_move] += 1
    last2 = tuple(opponent_history[-2:])
    if last2 not in transitions:
        return None, 0
    next_probs = transitions[last2]
    total = sum(next_probs.values())
    guess = max(next_probs, key=next_probs.get)
    confidence = next_probs[guess] / total
    return guess, confidence

ngram_cache = {}

def n_gram(n, source_history, target_history, actions, mode):
    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    strat_found = False
    pattern_counts = ngram_cache.setdefault((mode, n), defaultdict(lambda: defaultdict(int)))

    current_history = source_history if mode == 'Player' else target_history
    current_pattern = tuple(current_history[-n:])

    # Build pattern counts once
    if len(pattern_counts) == 0 and len(current_history) > n:
        for i in range(len(current_history) - n):
            pattern = tuple(current_history[i:i + n])
            next_move = current_history[i + n]
            pattern_counts[pattern][next_move] += 1

    total = sum(pattern_counts[current_pattern].values())
    if total == 0:
        return None, {}, 0, False

    probs = {a: pattern_counts[current_pattern].get(a, 0) / total for a in actions}
    guess = max(probs, key=probs.get)
    weight = (n / 20) * (total / (len(current_history) + 1))

    # Check if opponent is countering the pattern
    if len(target_history) > 0:
        player_apply_n_gram = 0
        for i in range(n + 1, len(target_history)):
            past_slice = current_history[:i]
            cur_pat = tuple(past_slice[-n:])
            if len(past_slice) <= n: continue

            count_dict = defaultdict(lambda: defaultdict(int))
            for j in range(len(past_slice) - n):
                pat = tuple(past_slice[j:j + n])
                nxt = past_slice[j + n]
                count_dict[pat][nxt] += 1

            if cur_pat in count_dict:
                most_likely = max(count_dict[cur_pat], key=count_dict[cur_pat].get)
                if counter[most_likely] == source_history[i]:
                    player_apply_n_gram += 1

        prob_stat = player_apply_n_gram / len(target_history)
        if prob_stat > 0.8:
            guess = counter[counter[guess]]
            strat_found = True

    return guess, probs, weight, strat_found


def detect_delayed_counter(My_history, opponent_history):
    max_lag = round(len(My_history)/2)
    if len(My_history) < max_lag + 5:
        return None

    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    best_lag = None
    best_match_score = 0

    for lag in range(1, max_lag + 1):
        match_count = 0
        comparisons = 0
        for i in range(lag, len(My_history)):
            expected = counter[My_history[i - lag]]
            if opponent_history[i] == expected:
                match_count += 1
            comparisons += 1

        if comparisons == 0:
            match_score = 0
        else:
            match_score = match_count / comparisons

        if match_score > best_match_score:
            best_match_score = match_score
            best_lag = lag

    if best_match_score > 0.80:
        return best_lag
    return None



def player(prev_play, opponent_history=[]):

    global My_history

    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    if prev_play == "":
        My_history = []
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}


        lag = detect_delayed_counter(My_history, opponent_history)
        if lag != None:
          Opponent_guess = My_history[-lag]
          Opponent_Counter_play = counter[Opponent_guess]
          counter_play = counter[Opponent_Counter_play]
          My_history.append(counter_play)
          return counter_play


        for i in range(1, min(round(m), 25)):
            if i == 2:
              boost = 1.2
            else:
              boost = 1.0

            guess_Player_history, Probability_Of_Events_PH, weight_PH, Strat_found_PH= n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probability_Of_Events_MH, weight_MH, Strat_found_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probability_Of_Events_PH, actions)
            confidence_level_MH = entropy(Probability_Of_Events_MH, actions)

            if Strat_found_MH == True:
              counter_play = guess_My_history
              My_history.append(counter_play)
              return counter_play

            if confidence_level_MH < confidence_level_PH:
              confidence_level = confidence_level_PH
              guess = guess_Player_history
              Type = 'Player'
              weighted_confidence_level = weight_PH * confidence_level * boost
            else:
              confidence_level = confidence_level_MH
              Opponent_guessing_me = guess_My_history
              if Opponent_guessing_me == None:
                  guess = None
              else:
                  guess = counter[Opponent_guessing_me]
              Type = 'Me'
              weighted_confidence_level = weight_MH * confidence_level * boost

            if guess != None and guess != '':
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))

        #markov_guess, markov_conf = markov_2gram(opponent_history)

        #if markov_guess and markov_conf > 0.3:
          #predicted_optimal_guesses[markov_guess] += markov_conf * 0.5

        #if markov_guess and markov_conf > 0.3 and predicted_optimal_guesses.get(markov_guess, 0) > 0:
          #predicted_optimal_guesses[markov_guess] += 0.3


        frequency = Counter(My_history)
        most_frequent = max(frequency, key=frequency.get)
        predicted_optimal_guesses[most_frequent] *= 0.80

        if len(My_history) >=3:
          if My_history[-1] == My_history[-3] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.5
          elif  My_history[-1] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.8
          else:
            predicted_optimal_guesses[counter[My_history[-1]]] += 0.1


        for play in actions:
          if predicted_optimal_guesses[play] < 0:
            predicted_optimal_guesses[play] = 0

        count = 0
        for values in predicted_optimal_guesses.values():
          if values == 0:
            count += 1
        if count == 3 or len(opponent_history) < 25:
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)


        counter_play = counter[predicted_move]
        remaining_actions = actions.copy()
        remaining_actions.remove(counter_play)
        if random.random() < 0.4:
           counter_play = random.choice(remaining_actions)


    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)

    return counter_play

In [ ]:
My_history = []
opponent_history = []

ngram_cache = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
log_lookup = [0.0] + [math.log2(1 + i / 100.0) for i in range(1, 101)]
cached_lag = None


def entropy(prob_events, actions):
    if len(set(prob_events.values())) <= 1:
        return 0
    entropy_val = -sum(p * math.log2(p) for p in prob_events.values() if p > 0)
    max_entropy = math.log2(len(actions))
    return 0 if max_entropy == 0 else 1 - entropy_val / max_entropy

def markov_2gram(opponent_history):
    if len(opponent_history) < 3:
        return None, 0
    transitions = defaultdict(lambda: defaultdict(int))
    for i in range(2, len(opponent_history)):
        prev2 = tuple(opponent_history[i-2:i])
        next_move = opponent_history[i]
        transitions[prev2][next_move] += 1
    last2 = tuple(opponent_history[-2:])
    if last2 not in transitions:
        return None, 0
    next_probs = transitions[last2]
    total = sum(next_probs.values())
    guess = max(next_probs, key=next_probs.get)
    confidence = next_probs[guess] / total
    return guess, confidence

def update_ngram_cache(history, n, mode):
    key = (mode, n)
    for i in range(len(history) - n):
        pattern = tuple(history[i:i + n])
        next_move = history[i + n]
        ngram_cache[key][pattern][next_move] += 1


def n_gram(n, source_history, target_history, actions, mode):
    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    key = (mode, n)
    pattern_counts = ngram_cache[key]

    current_history = source_history if mode == 'Player' else target_history
    if len(current_history) < n:
        return None, {}, 0, False

    current_pattern = tuple(current_history[-n:])
    total = sum(pattern_counts[current_pattern].values())
    if total == 0:
        return None, {}, 0, False

    probs = {a: pattern_counts[current_pattern].get(a, 0) / total for a in actions}
    guess = max(probs, key=probs.get)
    weight = (n / 20) * (total / (len(current_history) + 1))

    return guess, probs, weight, False


def detect_delayed_counter(my_hist, opp_hist):
    max_lag = round(len(my_hist) / 2)
    if len(my_hist) < max_lag + 5:
        return None

    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    best_lag = None
    best_score = 0

    for lag in range(1, max_lag + 1):
        match = sum(1 for i in range(lag, len(my_hist))
                    if opp_hist[i] == counter[my_hist[i - lag]])
        score = match / (len(my_hist) - lag)
        if score > best_score:
            best_score = score
            best_lag = lag

    return best_lag if best_score > 0.80 else None


def player(prev_play, opponent_history_local=[]):
    global My_history, opponent_history, cached_lag

    actions = ['R', 'P', 'S']
    counter = {'R': 'P', 'P': 'S', 'S': 'R'}

    if prev_play == "":
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    for n in range(1, min(11, len(opponent_history))):
        update_ngram_cache(opponent_history, n, 'Player')
        update_ngram_cache(My_history, n, 'Me')

    if len(opponent_history) > 2:
        n = len(opponent_history)
        m = int(n / 3)
        predicted_optimal_guesses = {'R': 0, 'P': 0, 'S': 0}

        if len(opponent_history) % 5 == 0:
            cached_lag = detect_delayed_counter(My_history, opponent_history)

        if cached_lag is not None:
            guess = My_history[-cached_lag]
            counter_play = counter[counter[guess]]
            My_history.append(counter_play)
            return counter_play

        for i in range(1, min(round(m), 11)):
            boost = 1.2 if i == 2 else 1.0

            guess_PH, probs_PH, weight_PH, _ = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_MH, probs_MH, weight_MH, _ = n_gram(i, opponent_history, My_history, actions, 'Me')

            conf_PH = entropy(probs_PH, actions)
            conf_MH = entropy(probs_MH, actions)

            if conf_MH < conf_PH:
                guess = guess_PH
                weighted_conf = weight_PH * conf_PH * boost
            else:
                guess = counter[guess_MH] if guess_MH else None
                weighted_conf = weight_MH * conf_MH * boost

            if guess:
                index = int(min(100, max(1, weighted_conf * 100)))
                predicted_optimal_guesses[guess] += log_lookup[index]


        markov_guess, markov_conf = markov_2gram(opponent_history)

        if markov_guess and markov_conf > 0.3:
          predicted_optimal_guesses[markov_guess] += markov_conf * 0.5

        if markov_guess and markov_conf > 0.3 and predicted_optimal_guesses.get(markov_guess, 0) > 0:
          predicted_optimal_guesses[markov_guess] += 0.3

        #Post-processing
        if My_history:
            freq = Counter(My_history)
            most_common = max(freq, key=freq.get)
            predicted_optimal_guesses[most_common] *= 0.80

            if len(My_history) >= 3:
                if My_history[-1] == My_history[-2] == My_history[-3]:
                    predicted_optimal_guesses[My_history[-1]] *= 0.5
                elif My_history[-1] == My_history[-2]:
                    predicted_optimal_guesses[My_history[-1]] *= 0.8
                else:
                    predicted_optimal_guesses[counter[My_history[-1]]] += 0.1

        # Final guess
        nonzero = sum(1 for v in predicted_optimal_guesses.values() if v > 0)
        if nonzero == 0 or len(opponent_history) < 25:
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)

        counter_play = counter[predicted_move]
        if random.random() < 0.4:
            counter_play = random.choice([a for a in actions if a != counter_play])

    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)
    return counter_play

In [ ]:
#RPS_game.py
# DO NOT MODIFY THIS FILE

def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])


In [ ]:

class UnitTests(unittest.TestCase):
    print()

    def test_player_vs_quincy(self):
        print("Testing game against quincy...")
        actual = play(player, quincy, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat quincy at least 60% of the time.')

    def test_player_vs_abbey(self):
        print("Testing game against abbey...")
        actual = play(player, abbey, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat abbey at least 60% of the time.')

    def test_player_vs_kris(self):
        print("Testing game against kris...")
        actual = play(player, kris, 1000) >= 60
        self.assertTrue(
            actual, 'Expected player to defeat kris at least 60% of the time.')

    def test_player_vs_mrugesh(self):
        print("Testing game against mrugesh...")
        actual = play(player, mrugesh, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat mrugesh at least 60% of the time.')


unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(UnitTests))

E


Testing game against abbey...
Testing game against kris...


.

Final results: {'p1': 993, 'p2': 5, 'tie': 2}
Player 1 win rate: 99.49899799599199%
Testing game against mrugesh...


.

Final results: {'p1': 467, 'p2': 271, 'tie': 262}
Player 1 win rate: 63.27913279132791%
Testing game against quincy...


.
ERROR: test_player_vs_abbey (__main__.UnitTests.test_player_vs_abbey)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-3867431552.py", line 13, in test_player_vs_abbey
    actual = play(player, abbey, 1000) >= 60
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1441766414.py", line 11, in play
    p2_play = player2(p1_prev_play)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1441766414.py", line 90, in abbey
    last_two = "".join(opponent_history[-2:])
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: sequence item 0: expected str instance, list found

----------------------------------------------------------------------
Ran 4 tests in 23.742s

FAILED (errors=1)


Final results: {'p1': 583, 'p2': 217, 'tie': 200}
Player 1 win rate: 72.875%


<unittest.runner.TextTestResult run=4 errors=1 failures=0>

In [ ]:
##Final

My_history = []
opponent_history = []
def entropy(Probibility_Of_Events, actions):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          Max_entropy = math.log2(len(actions))
          if Max_entropy == 0:
            confidence_level = 0
          else:
            confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def markov_ngram(opponent_history, n=2):
    if len(opponent_history) < n + 1:
        return None, 0
    transitions = defaultdict(lambda: defaultdict(int))
    for i in range(n, len(opponent_history)):
        prev = tuple(opponent_history[i - n:i])
        next_move = opponent_history[i]
        transitions[prev][next_move] += 1

    last = tuple(opponent_history[-n:])
    if last not in transitions:
        return None, 0

    next_probs = transitions[last]
    total = sum(next_probs.values())
    guess = max(next_probs, key=next_probs.get)
    confidence = next_probs[guess] / total
    return guess, confidence

def n_gram(n, opponent_history, My_history, actions, Type):
        counter = {'R':'P', 'P': 'S','S':'R'}
        dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        if Type == 'Player':
              Current_pattern = tuple(opponent_history[-n:])
              decay_rate = 0.99  # exponential decay
              for i in range(len(opponent_history) - n):
                pattern = tuple(opponent_history[i:i+n])
                players_next_play = opponent_history[i+n]
                weight = decay_rate ** (len(opponent_history) - i)  # more recent = higher weight
                dictionary_Of_Events[pattern][players_next_play] += weight
              history = opponent_history

        elif Type =='Me':
            Current_pattern = tuple(My_history[-n:])
            #for i in range(len(opponent_history)- n):
                #pattern = tuple(My_history[i:i+n])
                #players_next_play = opponent_history[i+n]
                #dictionary_Of_Events[pattern][players_next_play] += 1

            for i in range(len(My_history)- n):
                pattern = tuple(My_history[i:i+n])
                players_next_play = My_history[i+n]
                dictionary_Of_Events[pattern][players_next_play] += 1
            history = My_history

        for i in actions:
            if dictionary_Of_Events[Current_pattern].get(i) == None:
                dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(dictionary_Of_Events[Current_pattern].values())
        Probibility_Of_Events = {}
        if total_sum == 0:
          weight = 0
          guess = None
        else:
          for i in actions:
              Probibility_Of_Events[i] = dictionary_Of_Events[Current_pattern][i]/total_sum

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

          weight = (n / 5) * (total_sum / (len(history) + 1))


        return guess, Probibility_Of_Events, weight


def detect_delayed_counter(My_history, opponent_history):
    max_lag = round(len(My_history)/2)
    if len(My_history) < 10:
        return None

    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    best_lag = None
    best_match_score = 0

    for lag in range(1, max_lag + 1):
        match_count = 0
        comparisons = 0
        for i in range(lag, len(My_history)):
            expected = counter[My_history[i - lag]]
            if opponent_history[i] == expected:
                match_count += 1
            comparisons += 1

        if comparisons == 0:
            match_score = 0
        else:
            match_score = match_count / comparisons

        if match_score > best_match_score:
            best_match_score = match_score
            best_lag = lag

    if best_match_score > 0.80:
        return best_lag
    return None



def player(prev_play, opponent_history=[]):

    global My_history

    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    if prev_play == "":
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}


        lag = detect_delayed_counter(My_history, opponent_history)
        if lag != None:
          Opponent_guess = My_history[-lag]
          Opponent_Counter_play = counter[Opponent_guess]
          counter_play = counter[Opponent_Counter_play]
          My_history.append(counter_play)
          return counter_play

        confidence_level_MH = 0
        confidence_level_PH = 0
        for i in range(1, min(round(m),7)):
            boost = 1.0 + (i / (m + 1))  # Scales smoothly up to ~1.3
            boost = min(boost, 1.3)

            guess_Player_history, Probibility_Of_Events_PH, weight_PH = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probibility_Of_Events_MH, weight_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probibility_Of_Events_PH, actions)
            confidence_level_MH = entropy(Probibility_Of_Events_MH, actions)

            if confidence_level_MH < confidence_level_PH:
              confidence_level = confidence_level_PH
              guess = guess_Player_history
              Type = 'Player'
              weighted_confidence_level = weight_PH * confidence_level * boost
            else:
              confidence_level = confidence_level_MH
              Opponent_guessing_me = guess_My_history
              if Opponent_guessing_me == None:
                  guess = None
              else:
                  guess = counter[Opponent_guessing_me]
              Type = 'Me'
              weighted_confidence_level = weight_MH * confidence_level * boost

            if guess != None and guess != '':
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))

        for n in [2, 3]:
          markov_guess, markov_conf = markov_ngram(opponent_history, n=n)
          if markov_guess and markov_conf > 0.3:
            predicted_optimal_guesses[markov_guess] += markov_conf * (0.4 + 0.2 * n)  # slightly boost higher n


        frequency = {}
        frequency = defaultdict(lambda: defaultdict(int))
        for play in actions:
          frequency[play] = My_history.count(play)
        most_frequent = max(frequency, key=frequency.get)
        predicted_optimal_guesses[most_frequent] *= 0.80

        if len(My_history) >=3:
          if My_history[-1] == My_history[-3] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.5
          elif  My_history[-1] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.8
          else:
            predicted_optimal_guesses[counter[My_history[-1]]] += 0.1


        for play in actions:
          if predicted_optimal_guesses[play] < 0:
            predicted_optimal_guesses[play] = 0

        count = 0
        for values in predicted_optimal_guesses.values():
          if values == 0:
            count += 1
        if count == 3 or len(opponent_history) < 25:
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)

        if len(My_history) > 5:
            my_last = My_history[-1]
            beaten = {'R': 'P', 'P': 'S', 'S': 'R'}
            loss_count = sum(1 for i in range(-5, 0) if opponent_history[i] == beaten[My_history[i]])
            if loss_count > 3:
                counter_play = random.choice(actions)

        counter_play = counter[predicted_move]
        remaining_actions = actions
        remaining_actions.remove(counter_play)

        combined_entropy = (confidence_level_MH + confidence_level_PH) / 2
        jitter_threshold = 0.3 * (1 - combined_entropy)
        if random.random() < jitter_threshold:
            counter_play = random.choice([a for a in actions if a != predicted_move])
        #if combined_entropy < 0.4 and random.random() < 0.2:
            #counter_play = random.choice([a for a in actions if a != predicted_move])

    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)

    return counter_play

In [ ]:
#main.py

# This entrypoint file to be used in development. Start by reading README.md
#from RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
#from RPS import player

#play(player, quincy, 1000)
play(player, abbey, 1000)
play(player, abbey, 1000)
play(player, abbey, 1000)
play(player, abbey, 1000)
play(player, abbey, 1000)
play(player, abbey, 1000)
play(player, abbey, 1000)
play(player, abbey, 1000)
#play(player, kris, 1000)
#play(player, mrugesh, 1000)
#print(My_history)

# Uncomment line below to play interactively against a bot:
#play(human, abbey, 20, verbose=True)

# Uncomment line below to play against a bot that plays randomly:
# play(human, random_player, 1000)



# Uncomment line below to run unit tests automatically
#main(module='test_module', exit=False)#

UnboundLocalError: cannot access local variable 'Previous_Dictionary_Of_Events' where it is not associated with a value

In [ ]:
My_history = []
opponent_history = []
def entropy(Probibility_Of_Events, actions):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  #note - Why log2 used?
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          Max_entropy = math.log2(len(actions))
          if Max_entropy == 0:
            confidence_level = 0
          else:
            confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def n_gram(n, opponent_history, My_history, actions, Type):
        counter = {'R':'P', 'P': 'S','S':'R'}
        dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        if Type == 'Player':
              Current_pattern = tuple(opponent_history[-n:])
              #note - look at decay rate
              decay_rate = 0.99
              for i in range(len(opponent_history) - n):
                pattern = tuple(opponent_history[i:i+n])
                players_next_play = opponent_history[i+n]
                weight = decay_rate ** (len(opponent_history) - i)
                dictionary_Of_Events[pattern][players_next_play] += weight
              history = opponent_history

        elif Type =='Me':
            Current_pattern = tuple(My_history[-n:])
            for i in range(len(My_history)- n):
                pattern = tuple(My_history[i:i+n])
                players_next_play = My_history[i+n]
                dictionary_Of_Events[pattern][players_next_play] += 1
            history = My_history

        for i in actions:
            if dictionary_Of_Events[Current_pattern].get(i) == None:
                dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(dictionary_Of_Events[Current_pattern].values())
        Probibility_Of_Events = {}
        if total_sum == 0:
          weight = 0
          guess = None
        else:
          for i in actions:
              Probibility_Of_Events[i] = dictionary_Of_Events[Current_pattern][i]/total_sum

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

          #note - check weighting
          #How accurate the guess is?
          #note - check if significantly higher weight compared to other n-grams
          weight = (n / 5) * (total_sum / (len(history) + 1))


        return guess, Probibility_Of_Events, weight


def detect_delayed_counter(My_history, opponent_history):
    max_lag = round(len(My_history)/2)
    #note - A little random, e.g., 10, can you calculate a better value?
    if len(My_history) < 10:
        return None

    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    best_lag = None
    best_match_score = 0

    for lag in range(1, max_lag + 1):
        match_count = 0
        comparisons = 0
        for i in range(lag, len(My_history)):
            expected = counter[My_history[i - lag]]
            if opponent_history[i] == expected:
                match_count += 1
            comparisons += 1
        #less evidence for higher lags hence we need to take that into account by dividing by comparisons
        #note - may need to consider creating a weighting for the number of comparison,s as fewer comparisons mean less accurate
        match_score = match_count / comparisons

        if match_score > best_match_score:
            best_match_score = match_score
            best_lag = lag

    if best_match_score > 0.80:
        return best_lag
    return None


def player(prev_play, opponent_history=[]):
    #note - reduce computation time by calculating the frequency of events and returning it back from previsous functions and only compute admendments to the
    #frequency dictionary once each round, based on the new influx of data.

    global My_history

    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    if not prev_play:
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}


        lag = detect_delayed_counter(My_history, opponent_history)
        if lag != None:
          Opponent_guess = My_history[-lag]
          Opponent_Counter_play = counter[Opponent_guess]
          counter_play = counter[Opponent_Counter_play]
          My_history.append(counter_play)
          return counter_play

        confidence_level_MH = 0
        confidence_level_PH = 0
        for i in range(1, min(round(m),7)):
          #note - boost seems quite arbitrary
            boost = 1.0 + (i / (m + 1))
            boost = min(boost, 1.3)

            guess_Player_history, Probibility_Of_Events_PH, weight_PH = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probibility_Of_Events_MH, weight_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probibility_Of_Events_PH, actions)
            confidence_level_MH = entropy(Probibility_Of_Events_MH, actions)
            #note - compare weight to other n-gram weights see if there is an obvious n-gram strategy conducted by the opponent
            if confidence_level_MH < confidence_level_PH:
              confidence_level = confidence_level_PH
              guess = guess_Player_history
              Type = 'Player'
              weighted_confidence_level = weight_PH * confidence_level * boost
            else:
              confidence_level = confidence_level_MH
              #Opponent guessing what my next move will be
              Opponent_guessing_me = guess_My_history
              if Opponent_guessing_me == None:
                  guess = None
              else:
                  # My prediction of what move the opponent will make
                  guess = counter[Opponent_guessing_me]
              Type = 'Me'
              weighted_confidence_level = weight_MH * confidence_level * boost

            #note - Why is there 2 possible null cases? Find and fix
            if guess != None and guess != '':
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))

        # Decreasing the weight of me choosing the most frequently used move to avoid playing too predictably
        frequency = {}
        frequency = defaultdict(lambda: defaultdict(int))
        for play in actions:
          frequency[play] = My_history.count(play)
        most_frequent = max(frequency, key=frequency.get)
        #note - compare frequency to others to see if there is a major disparity and only decrease the weight if this is the case
        predicted_optimal_guesses[most_frequent] *= 0.80
        #note - arbitary proportion of decrease, possibly look to choose a more fitting proportion thats more relevant

        #note - may not be nessary to reduce the weights in this case because then counter bots could learn that I dont play the same move lots of times in a row and hence
        # counter me. It is natural to have long strings of moves when playing randomly. Also, values are arbitrary, which doesn't make them very relevant and adds noise.
        if len(My_history) >=3:
          if My_history[-1] == My_history[-3] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.5
          elif  My_history[-1] == My_history[-2]:
            predicted_optimal_guesses[My_history[-1]] *= 0.8
          else:
            predicted_optimal_guesses[counter[My_history[-1]]] += 0.1

        #note - should try to avoid negatives from ever occuring by using muliplication of proportions instead of simple addition and subtraction in all cases
        for play in actions:
          if predicted_optimal_guesses[play] < 0:
            predicted_optimal_guesses[play] = 0

        #note - does this case occur that all values = 0? If not, consider removing it as it would be unnecessary.
        count = 0
        for values in predicted_optimal_guesses.values():
          if values == 0:
            count += 1
        #note - why 25? Quite arbitrary. Can you calculate a better limit?
        if count == 3 or len(opponent_history) < 25:
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)

        #note - Could be adding noise, too small of a range
        if len(My_history) > 5:
            my_last = My_history[-1]
            beaten = {'R': 'P', 'P': 'S', 'S': 'R'}
            loss_count = sum(1 for i in range(-5, 0) if opponent_history[i] == beaten[My_history[i]])
            if loss_count > 3:
                counter_play = random.choice(actions)

        counter_play = counter[predicted_move]
        remaining_actions = actions
        remaining_actions.remove(counter_play)

        #note - Perpose is to add noise to my moves, however may cause me to lose if my prediction was correct. possibly consider removing if it is
        #doing more harm than good.
        combined_entropy = (confidence_level_MH + confidence_level_PH) / 2
        jitter_threshold = 0.3 * (1 - combined_entropy)
        if random.random() < jitter_threshold:
            counter_play = random.choice([a for a in actions if a != predicted_move])

    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)
    #note - Focus on abbey and reducing computation time until we get the result needed
    #note - Research more about Markov chains to see if they would be a help

    return counter_play

In [ ]:
My_history = []
opponent_history = []
def entropy(Probibility_Of_Events):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  #note - Why log2 used?
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          if len(Probibility_Of_Events) == 0 or not Probibility_Of_Events:
              return 0
          else:
            Max_entropy = math.log2(len(Probibility_Of_Events))
            if Max_entropy == 0:
              confidence_level = 0
            else:
              confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def n_gram(n, opponent_history, My_history, actions, Type):
        counter = {'R':'P', 'P': 'S','S':'R'}
        if Type == 'Player':
              Current_pattern = tuple(opponent_history[-n:])
              #previous_pattern = tuple(opponent_history[-n-1:-1])
              #1
              pattern = tuple(opponent_history[-(n+1):-1])
              players_next_play = opponent_history[-1]
              Dictionary_Of_Events[pattern][players_next_play] += 1
              history = opponent_history

        elif Type =='Me':
             Current_pattern = tuple(My_history[-n:])
             #previous_pattern = tuple(My_history[-n-1:-1])
             #2
             pattern = tuple(My_history[-(n+1):-1])
             players_next_play = My_history[-1]
             Dictionary_Of_Events[pattern][players_next_play] += 1
             history = My_history

        for i in actions:
            if Dictionary_Of_Events[Current_pattern].get(i) == None:
                Dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(Dictionary_Of_Events[Current_pattern].values())
        Probibility_Of_Events = {}
        if total_sum == 0:
          weight = 0
          guess = None
        else:
          for i in actions:
              Probibility_Of_Events[i] = Dictionary_Of_Events[Current_pattern][i]/total_sum

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

          #note - check weighting
          #How accurate the guess is?
          #note - check if significantly higher weight compared to other n-grams
          # note - Weight calculation: A higher n_gram
          weight = (n / 5) * (total_sum / (len(history) + 1))

        if len(My_history) <= 10:
            correct_guess = 0
        else:

            correct_guess = 0
            try: guesses[n]
            except: guesses[n] = []
            if guesses[n]:
              opponents_possible_play = guesses[n][-1]
              #if Type == 'Me':
                  #opponents proposed play
                  #opponents_possible_play = counter[guesses[n][-1]]
              #else:
                  #opponents_possible_play = guesses[n][-1]

              if opponents_possible_play == opponent_history[-1]:
                  correct_guess += 1
                 #3

        return guess, Probibility_Of_Events, weight, correct_guess

        #return dictionary_Of_events

def detect_delayed_counter(My_history, opponent_history):
    max_lag = round(len(My_history)/2)
    #note - A little random, e.g., 10, can you calculate a better value?
    if len(My_history) < 10:
        return None

    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    best_lag = None
    best_match_score = 0

    for lag in range(1, max_lag + 1):
        match_count = 0
        comparisons = 0
        for i in range(lag, len(My_history)):
            expected = counter[My_history[i - lag]]
            if opponent_history[i] == expected:
                match_count += 1
            comparisons += 1
        #less evidence for higher lags hence we need to take that into account by dividing by comparisons
        #note - may need to consider creating a weighting for the number of comparison,s as fewer comparisons mean less accurate
        match_score = match_count / comparisons

        if match_score > best_match_score:
            best_match_score = match_score
            best_lag = lag

    if best_match_score > 0.80:
        return best_lag
    return None


def player(prev_play, opponent_history=[]):
    #note - reduce computation time by calculating the frequency of events and returning it back from previsous functions and only compute admendments to the
    #frequency dictionary once each round, based on the new influx of data.

    global My_history

    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    global counts
    global Dictionary_Of_Events
    global weights
    global guesses
    global guess_types

    if not prev_play:
        weights = {}
        guesses = {}
        guess_types = {}
        counts = {}
        Dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}


        lag = detect_delayed_counter(My_history, opponent_history)
        if lag != None:
          Opponent_guess = My_history[-lag]
          Opponent_Counter_play = counter[Opponent_guess]
          counter_play = counter[Opponent_Counter_play]
          My_history.append(counter_play)
          return counter_play

        confidence_level_MH = 0
        confidence_level_PH = 0
        Total_sum_PH_confidence = 0
        Total_sum_MH_confidence = 0
        strategy_detected = False
        highest_value = 0
        for i in range(1, min(round(m),7)):
          #note - boost seems quite arbitrary
            boost = 1.0 + (i / (m + 1))
            boost = min(boost, 1.3)

            guess_Player_history, Probibility_Of_Events_PH, weight_PH, correct_guess_PH = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probibility_Of_Events_MH, weight_MH, correct_guess_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probibility_Of_Events_PH)
            confidence_level_MH = entropy(Probibility_Of_Events_MH)
            #note - compare weight to other n-gram weights see if there is an obvious n-gram strategy conducted by the opponent
            if confidence_level_MH < confidence_level_PH:
              confidence_level = confidence_level_PH
              Total_sum_PH_confidence += confidence_level
              guess = guess_Player_history
              Type = 'Player'
              weighted_confidence_level = weight_PH * confidence_level * boost
              correct_guess = correct_guess_PH
            else:
              confidence_level = confidence_level_MH
              Total_sum_MH_confidence += confidence_level
              #Opponent guessing what my next move will be
              Opponent_guessing_me = guess_My_history
              if Opponent_guessing_me == None:
                  guess = None
              else:
                  # My prediction of what move the opponent will make
                  guess = counter[Opponent_guessing_me]
              Type = 'Me'
              weighted_confidence_level = weight_MH * confidence_level * boost
              correct_guess = correct_guess_MH

            #note - Why is there 2 possible null cases? Find and fix
            if (guess) and weighted_confidence_level != 0:
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))
                try: guesses[i].append(guess)
                except: guesses[i] = [guess]
                try: guess_types[i].append(Type)
                except: guess_types[i] = [Type]
                try: weights[i].append(confidence_level)
                except: weights[i] = [confidence_level]

                try: counts[i] += correct_guess
                except: counts[i] = correct_guess

            optimal_n_gram = None
            #if len(opponent_history) > 10:
             # if counts.get(i, 0)/(len(My_history) - 10) > 0.80 and counts.get(i, 0)/(len(My_history) - 10) > highest_value:
            if len(opponent_history) > 10:
                acc = counts.get(i, -1)/(len(My_history) - 10)
                #print(f"n={i}, accuracy={acc:.2f}")
                if acc > 0.35 and acc > highest_value:
                  highest_value = counts[i]/(len(My_history) - 10)
                  optimal_n_gram = i
                  predicted_guess = guess
                  strategy_detected = True



        if strategy_detected and optimal_n_gram is not None:
            print(f"Detected n-gram strategy: n={optimal_n_gram}, type={guess_types[optimal_n_gram][-1]}")
            #predicted_move = guesses[optimal_n_gram][-1]
            guess, Probibility_Of_Events, weight, correct_guess = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_type = guess_types[optimal_n_gram][-1]
            if guess_type == 'Me':
                counter_play = counter[counter[predicted_guess]]
            else:
                counter_play = counter[predicted_guess]
            My_history.append(counter_play)
            return counter_play


        if len(weights) == 0:
          counter_play = random.choice(actions)
          My_history.append(counter_play)
          return counter_play

          #4


          # Decreasing the weight of me choosing the most frequently used move to avoid playing too predictably
        frequency = {}
        #frequency = defaultdict(lambda: defaultdict(int))
        for play in actions:
          frequency[play] = My_history.count(play)
        confidence_freq_play = entropy(frequency)
        if confidence_freq_play > 0.80:
          most_frequent = max(frequency, key=frequency.get)
          predicted_optimal_guesses[most_frequent] *= 0.85
          #note - compare frequency to others to see if there is a major disparity and only decrease the weight if this is the case
          #note - arbitary proportion of decrease, possibly look to choose a more fitting proportion thats more relevant

          #note - does this case occur that all values = 0? If not, consider removing it as it would be unnecessary.
        count = 0
        for values in predicted_optimal_guesses.values():
          if values == 0:
            count += 1
          #note - why 25? Quite arbitrary. Can you calculate a better limit?
        if count == 3 or len(opponent_history) < 15:
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)

          #note - Could be adding noise, too small of a range
        if len(My_history) > 20:
            my_last = My_history[-1]
            beaten = {'R': 'P', 'P': 'S', 'S': 'R'}
            loss_count = 0
            for i in range(-20, 0):
                if opponent_history[i] == beaten[My_history[i]]:
                  loss_count += 1
            if loss_count > 16:
                counter_play = random.choice(actions)
                My_history.append(counter_play)
                return counter_play


        counter_play = counter[predicted_move]
        #remaining_actions = actions
        #remaining_actions.remove(counter_play)

        #note - Perpose is to add noise to my moves, however may cause me to lose if my prediction was correct. possibly consider removing if it is
        #doing more harm than good.
        average_MH_confidence = Total_sum_MH_confidence / min(round(m),7)
        average_PH_confidence = Total_sum_PH_confidence / min(round(m),7)
        combined_entropy = (average_MH_confidence + average_PH_confidence) / 2
        jitter_threshold = 0.2 * (1 - combined_entropy)
        if random.random() < jitter_threshold and combined_entropy < 0.75:
           counter_play = random.choice(actions)

    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)
    #note - Focus on abbey and reducing computation time until we get the result needed
    #note - Research more about Markov chains to see if they would be a help

    return counter_play

In [ ]:
#1
#note - look at decay rate
              #decay_rate = 0.99
              #for i in range(len(opponent_history) - n):
                #pattern = tuple(opponent_history[i:i+n])
                #players_next_play = opponent_history[i+n]
                #weight = decay_rate ** (len(opponent_history) - i)
                #dictionary_Of_Events[pattern][players_next_play] += weight
#2

#for i in range(len(My_history)- n):
                #pattern = tuple(My_history[i:i+n])
                #Opponents guess for what I will play
                #players_next_play = My_history[i+n]
                #dictionary_Of_Events[pattern][players_next_play] += 1
#3
#for i in range(10,len(My_history)):
              #My_history_copy = My_history[:i]
              #opponent_history_copy = opponent_history[:i]
              #guess, Probibility_Of_Events, weight = n_gram(n, opponent_history_copy, My_history_copy, actions, Type)
              #if guess == None:
                #continue
              #if Type == 'Me':
                #opponents proposed play
                #opponents_possible_play = counter[guess]
              #else:
                #opponents_possible_play = guess

              #if opponents_possible_play == opponent_history[i]:
                #correct_guess += 1

#4
        #for n in range(1, min(round(m),7)):
         #n_gram_strategy = detect_n_gram(n,opponent_history, My_history, 'Me', actions, counter)
          #n_gram_strategy_Player = detect_n_gram(n,opponent_history, My_history, 'Player', actions, counter)
          #if n_gram_strategy:
            #predicted_move = guesses[n]
            #counter_play = counter[predicted_move]
            #My_history.append(counter_play)
            #return counter_play


#combined_weight = sum(weights.values())
        #for i in weights:
          #weights[i] = weights[i] / combined_weight

        #overall_confidence_weights = entropy(weights)

        #if overall_confidence_weights > 0.70:
          #optimal_n_gram = max(weights, key=weights.get)
          #predicted_move = guesses[optimal_n_gram]
          #counter_play = counter[predicted_move]
          #My_history.append(counter_play)
          #return counter_play

        #ChatGTP suggestsion
        #sorted_weights = sorted(weights.items(), key=lambda x: x[1], reverse=True)
        #if len(sorted_weights) > 1:
          #best_n, best_val = sorted_weights[0]
          #second_val = sorted_weights[1][1]
          #gap = best_val - second_val

          #if best_val > 0.60 and gap > 0.15:
              #predicted_move = guesses[best_n]
              #counter_play = counter[predicted_move]
              #My_history.append(counter_play)
              #return counter_play

        #max_name = max(weights, key=weights.get)
        #max_value = weights[max_name]
        #weights[max] = 0
        #second = max(weights, key=weights.get)
        #second_value = weights[second]
        #if max_value > 0.60 and (max_value - second_value) > 0.1  :
          #predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)
          #counter_play = counter[counter[predicted_move]]
        #else:



        def detect_n_gram(n,opponent_history, My_history, Type, actions, counter):
  if len(My_history) <= 10:
    return False

  correct_guess = 0
  for i in range(10,len(My_history)):
      My_history_copy = My_history[:i]
      opponent_history_copy = opponent_history[:i]
      guess, Probibility_Of_Events, weight = n_gram(n, opponent_history_copy, My_history_copy, actions, Type)
      if guess == None:
        continue
      if Type == 'Me':
        #opponents proposed play
        opponents_possible_play = counter[guess]
      else:
        opponents_possible_play = guess

      if opponents_possible_play == opponent_history[i]:
        correct_guess += 1

  if correct_guess/(len(My_history) - 10) > 0.80:
    return True
  else:
    return False

In [ ]:
My_history = []
opponent_history = []
def entropy(Probibility_Of_Events):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  #note - Why log2 used?
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          if len(Probibility_Of_Events) == 0 or not Probibility_Of_Events:
              return 0
          else:
            Max_entropy = math.log2(len(Probibility_Of_Events))
            if Max_entropy == 0:
              confidence_level = 0
            else:
              confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def n_gram(n, opponent_history, My_history, actions, Type):
        counter = {'R':'P', 'P': 'S','S':'R'}
        if Type == 'Player':
              Current_pattern = tuple(opponent_history[-n:])
              #previous_pattern = tuple(opponent_history[-n-1:-1])
              #1
              pattern = tuple(opponent_history[-(n+1):-1])
              players_next_play = opponent_history[-1]
              Dictionary_Of_Events[pattern][players_next_play] += 1
              history = opponent_history

        elif Type =='Me':
             Current_pattern = tuple(My_history[-n:])
             #previous_pattern = tuple(My_history[-n-1:-1])
             #2
             pattern = tuple(My_history[-(n+1):-1])
             players_next_play = My_history[-1]
             Dictionary_Of_Events[pattern][players_next_play] += 1
             history = My_history

        for i in actions:
            if Dictionary_Of_Events[Current_pattern].get(i) == None:
                Dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(Dictionary_Of_Events[Current_pattern].values())
        Probibility_Of_Events = {}
        if total_sum == 0:
          weight = 0
          guess = None
        else:
          for i in actions:
              Probibility_Of_Events[i] = Dictionary_Of_Events[Current_pattern][i]/total_sum

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

          #note - check weighting
          #How accurate the guess is?
          #note - check if significantly higher weight compared to other n-grams
          # note - Weight calculation: A higher n_gram
          weight = (n / 5) * (total_sum / (len(history) + 1))

        if len(My_history) <= 10:
            correct_guess = 0
        else:

            correct_guess = 0
            try: guesses[n]
            except: guesses[n] = []
            if guesses[n]:
              opponents_possible_play = guesses[n][-1]
              #if Type == 'Me':
                  #opponents proposed play
                  #opponents_possible_play = counter[guesses[n][-1]]
              #else:
                  #opponents_possible_play = guesses[n][-1]

              if opponents_possible_play == opponent_history[-1]:
                  correct_guess += 1
                 #3

        return guess, Probibility_Of_Events, weight, correct_guess

        #return dictionary_Of_events

def detect_delayed_counter(My_history, opponent_history):
    max_lag = round(len(My_history)/2)
    #note - A little random, e.g., 10, can you calculate a better value?
    if len(My_history) < 10:
        return None

    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    best_lag = None
    best_match_score = 0

    for lag in range(1, max_lag + 1):
        match_count = 0
        comparisons = 0
        for i in range(lag, len(My_history)):
            expected = counter[My_history[i - lag]]
            if opponent_history[i] == expected:
                match_count += 1
            comparisons += 1
        #less evidence for higher lags hence we need to take that into account by dividing by comparisons
        #note - may need to consider creating a weighting for the number of comparison,s as fewer comparisons mean less accurate
        match_score = match_count / comparisons

        if match_score > best_match_score:
            best_match_score = match_score
            best_lag = lag

    if best_match_score > 0.80:
        return best_lag
    return None


def player(prev_play, opponent_history=[]):
    global My_history
    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    global counts
    global Dictionary_Of_Events
    global weights
    global guesses
    global guess_types

    if not prev_play:
        weights = {}
        guesses = {}
        guess_types = {}
        counts = {}
        Dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}

        lag = detect_delayed_counter(My_history, opponent_history)
        if lag != None:
            Opponent_guess = My_history[-lag]
            Opponent_Counter_play = counter[Opponent_guess]
            counter_play = counter[Opponent_Counter_play]
            My_history.append(counter_play)
            return counter_play

        confidence_level_MH = 0
        confidence_level_PH = 0
        Total_sum_PH_confidence = 0
        Total_sum_MH_confidence = 0
        strategy_detected = False
        highest_value = 0
        for i in range(1, min(round(m),7)):
            boost = 1.0 + (i / (m + 1))
            boost = min(boost, 1.3)

            guess_Player_history, Probility_Of_Events_PH, weight_PH, correct_guess_PH = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probility_Of_Events_MH, weight_MH, correct_guess_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probility_Of_Events_PH)
            confidence_level_MH = entropy(Probility_Of_Events_MH)

            if confidence_level_MH < confidence_level_PH:
                confidence_level = confidence_level_PH
                Total_sum_PH_confidence += confidence_level
                guess = guess_Player_history
                Type = 'Player'
                weighted_confidence_level = weight_PH * confidence_level * boost
                correct_guess = correct_guess_PH
            else:
                confidence_level = confidence_level_MH
                Total_sum_MH_confidence += confidence_level
                Opponent_guessing_me = guess_My_history
                if Opponent_guessing_me is None:
                    guess = None
                else:
                    # NOTE: Abbey predicts your next move and plays the counter
                    # You want to play the predicted move itself to beat Abbey's counter move
                    guess = Opponent_guessing_me
                Type = 'Me'
                weighted_confidence_level = weight_MH * confidence_level * boost
                correct_guess = correct_guess_MH

            if guess and weighted_confidence_level != 0:
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))
                guesses.setdefault(i, []).append(guess)
                guess_types.setdefault(i, []).append(Type)
                weights.setdefault(i, []).append(confidence_level)
                counts[i] = counts.get(i, 0) + correct_guess

            if len(opponent_history) > 10:
                acc = counts.get(i, 0) / (len(My_history) - 10)
                if acc > 0.35 and acc > highest_value:
                    highest_value = acc
                    optimal_n_gram = i
                    strategy_detected = True

        if strategy_detected and optimal_n_gram is not None:
            #print(f"Detected n-gram strategy: n={optimal_n_gram}, type={guess_types[optimal_n_gram][-1]}")
            predicted_move = guesses[optimal_n_gram][-1]
            guess_type = guess_types[optimal_n_gram][-1]
            # IMPORTANT CHANGE:
            # Abbey plays counter(predicted_move)
            # You want to play predicted_move to beat Abbey's counter
            My_history.append(predicted_move)
            return predicted_move

        if len(weights) == 0:
            counter_play = random.choice(actions)
            My_history.append(counter_play)
            return counter_play

        frequency = {play: My_history.count(play) for play in actions}
        confidence_freq_play = entropy(frequency)
        if confidence_freq_play > 0.80:
            most_frequent = max(frequency, key=frequency.get)
            predicted_optimal_guesses[most_frequent] *= 0.85

        count = sum(1 for v in predicted_optimal_guesses.values() if v == 0)
        if count == 3 or len(opponent_history) < 15:
            predicted_move = random.choice(actions)
        else:
            predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)

        if len(My_history) > 20:
            my_last = My_history[-1]
            beaten = {'R': 'P', 'P': 'S', 'S': 'R'}
            loss_count = 0
            for i in range(-20, 0):
                if opponent_history[i] == beaten[My_history[i]]:
                    loss_count += 1
            if loss_count > 16:
                counter_play = random.choice(actions)
                My_history.append(counter_play)
                return counter_play

        # FINAL MOVE:
        # Abbey plays counter(predicted_move), so you should play predicted_move
        My_history.append(predicted_move)
        return predicted_move

    else:
        counter_play = random.choice(actions)
        My_history.append(counter_play)
        return counter_play

In [ ]:
#Bot against abbey
My_history = []
opponent_history = []
def entropy(Probibility_Of_Events):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  #note - Why log2 used?
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          if len(Probibility_Of_Events) == 0 or not Probibility_Of_Events:
              return 0
          else:
            Max_entropy = math.log2(len(Probibility_Of_Events))
            if Max_entropy == 0:
              confidence_level = 0
            else:
              confidence_level = 1 - entropy/Max_entropy
          return confidence_level

def abbey(opponents_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not opponents_history:
        prev_opponent_play = 'R'
    else:
        prev_opponent_play = opponents_history[-1]

    last_two = "".join(opponents_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]

def n_gram(n, opponent_history, My_history, actions, Type):
        correct_guess = 0
        counter = {'R':'P', 'P': 'S','S':'R'}
        if Type == 'Player':
              Current_pattern = tuple(opponent_history[-n:])
              #previous_pattern = tuple(opponent_history[-n-1:-1])
              #1
              pattern = tuple(opponent_history[-(n+1):-1])
              players_next_play = opponent_history[-1]
              Dictionary_Of_Events[pattern][players_next_play] += 1
              history = opponent_history

        elif Type =='Me':
             Current_pattern = tuple(My_history[-n:])
             #previous_pattern = tuple(My_history[-n-1:-1])
             #2
             pattern = tuple(My_history[-(n+1):-1])
             players_next_play = My_history[-1]
             Dictionary_Of_Events[pattern][players_next_play] += 1
             history = My_history

        for i in actions:
            if Dictionary_Of_Events[Current_pattern].get(i) == None:
                Dictionary_Of_Events[Current_pattern][i] = 0

        total_sum = sum(Dictionary_Of_Events[Current_pattern].values())
        Probibility_Of_Events = {}
        if total_sum == 0:
          weight = 0
          guess = None
        else:
          for i in actions:
              Probibility_Of_Events[i] = Dictionary_Of_Events[Current_pattern][i]/total_sum

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

          #note - check weighting
          #How accurate the guess is?
          #note - check if significantly higher weight compared to other n-grams
          # note - Weight calculation: A higher n_gram
          weight = (n / 5) * (total_sum / (len(history) + 1))

          abbey_counter_play = abbey(My_history, play_order)
          if counter[guess] == abbey_counter_play:
            correct_guess += 1
          else:
            correct_guess = 0


        return guess, Probibility_Of_Events, weight, correct_guess

        #return dictionary_Of_events




def player(prev_play, opponent_history=[]):
    #note - reduce computation time by calculating the frequency of events and returning it back from previsous functions and only compute admendments to the
    #frequency dictionary once each round, based on the new influx of data.

    global My_history

    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    global counts
    global Dictionary_Of_Events
    global weights
    global guesses
    global guess_types

    if not prev_play:
        weights = {}
        guesses = {}
        guess_types = {}
        counts = {}
        Dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}


        lag = detect_delayed_counter(My_history, opponent_history)
        if lag != None:
          Opponent_guess = My_history[-lag]
          Opponent_Counter_play = counter[Opponent_guess]
          counter_play = counter[Opponent_Counter_play]
          My_history.append(counter_play)
          return counter_play

        confidence_level_MH = 0
        confidence_level_PH = 0
        Total_sum_PH_confidence = 0
        Total_sum_MH_confidence = 0
        strategy_detected = False
        highest_value = 0
        for i in range(1,2):
          #note - boost seems quite arbitrary
            boost = 1.0 + (i / (m + 1))
            boost = min(boost, 1.3)

            guess_Player_history, Probibility_Of_Events_PH, weight_PH, correct_guess_PH = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_My_history, Probibility_Of_Events_MH, weight_MH, correct_guess_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_PH = entropy(Probibility_Of_Events_PH)
            confidence_level_MH = entropy(Probibility_Of_Events_MH)
            #note - compare weight to other n-gram weights see if there is an obvious n-gram strategy conducted by the opponent
            if confidence_level_MH < confidence_level_PH:
              confidence_level = confidence_level_PH
              Total_sum_PH_confidence += confidence_level
              guess = guess_Player_history
              Type = 'Player'
              weighted_confidence_level = weight_PH * confidence_level * boost
              correct_guess = correct_guess_PH
            else:
              confidence_level = confidence_level_MH
              Total_sum_MH_confidence += confidence_level
              #Opponent guessing what my next move will be
              Opponent_guessing_me = guess_My_history
              if Opponent_guessing_me == None:
                  guess = None
              else:
                  # My prediction of what move the opponent will make
                  guess = counter[Opponent_guessing_me]
              Type = 'Me'
              weighted_confidence_level = weight_MH * confidence_level * boost
              correct_guess = correct_guess_MH

            #note - Why is there 2 possible null cases? Find and fix
            if (guess) and weighted_confidence_level != 0:
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))
                try: guesses[i].append(guess)
                except: guesses[i] = [guess]
                try: guess_types[i].append(Type)
                except: guess_types[i] = [Type]
                try: weights[i].append(confidence_level)
                except: weights[i] = [confidence_level]

                try: counts[i] += correct_guess
                except: counts[i] = correct_guess


            optimal_n_gram = None
            if counts[i]/len(My_history) > 0.80:
                  print(f"n={i}, accuracy={counts[i]/len(My_history):.2f}")
                  optimal_n_gram = i
                  predicted_guess = guess
                  strategy_detected = True



        if strategy_detected and optimal_n_gram is not None:
            print(f"Detected n-gram strategy: n={optimal_n_gram}, type={guess_types[optimal_n_gram][-1]}")
            #predicted_move = guesses[optimal_n_gram][-1]
            guess, Probibility_Of_Events, weight, correct_guess = n_gram(i, opponent_history, My_history, actions, 'Player')
            guess_type = guess_types[optimal_n_gram][-1]
            if guess_type == 'Me':
                counter_play = counter[counter[predicted_guess]]
            else:
                counter_play = counter[predicted_guess]
            My_history.append(counter_play)
            return counter_play


        if len(weights) == 0:
          counter_play = random.choice(actions)
          My_history.append(counter_play)
          return counter_play



        counter_play = counter[predicted_move]


    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)
    #note - Focus on abbey and reducing computation time until we get the result needed
    #note - Research more about Markov chains to see if they would be a help

    return counter_play


In [ ]:
import math
import random
from collections import defaultdict

# Global histories
My_history = []
opponent_history = []

def entropy(Probibility_Of_Events):
    entropy = 0
    for probibility in Probibility_Of_Events:
        if Probibility_Of_Events[probibility] != 0:
            entropy -= Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
    if len(Probibility_Of_Events) == 0:
        return 0
    else:
        Max_entropy = math.log2(len(Probibility_Of_Events))
        confidence_level = 0 if Max_entropy == 0 else 1 - entropy / Max_entropy
        return confidence_level

def abbey(opponents_My_history=None):

    if opponents_My_history is None:
        opponents_My_history = []
    if len(opponents_My_history) < 1:
        return 'R'
    last_two = "".join(opponents_My_history[-2:])
    if len(last_two) == 2:
        play_order[last_two] += 1

    potential_plays = [ opponents_My_history[-1] + move for move in "RPS"]
    sub_order = {k: play_order[k] for k in potential_plays if k in play_order}

    prediction = max(sub_order, key=sub_order.get)[-1:] if sub_order else 'R'
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction], play_order

def n_gram(n, opponent_history, My_history, actions, Type):
    counter = {'R':'P', 'P': 'S','S':'R'}
    correct_guess = 0
    if Type == 'Player':
        Current_pattern = tuple(opponent_history[-n:])
        pattern = tuple(opponent_history[-(n+1):-1])
        players_next_play = opponent_history[-1]
        Dictionary_Of_Events[pattern][players_next_play] += 1
        history = opponent_history
    elif Type == 'Me':
        Current_pattern = tuple(My_history[-n:])
        pattern = tuple(My_history[-(n+1):-1])
        players_next_play = My_history[-1]
        Dictionary_Of_Events[pattern][players_next_play] += 1
        history = My_history

    for i in actions:
        Dictionary_Of_Events[Current_pattern][i] = Dictionary_Of_Events[Current_pattern].get(i, 0)

    total_sum = sum(Dictionary_Of_Events[Current_pattern].values())
    Probibility_Of_Events = {}
    if total_sum == 0:
        weight = 0
        guess = None
    else:
        for i in actions:
            Probibility_Of_Events[i] = Dictionary_Of_Events[Current_pattern][i] / total_sum
        guess = max(Probibility_Of_Events, key=Probibility_Of_Events.get)
        weight = (n / 5) * (total_sum / (len(history) + 1))

        abbey_counter_play, returned_play_order = abbey(My_history.copy())
        if guess and counter[guess] == abbey_counter_play:
            correct_guess += 1

    return guess, Probibility_Of_Events, weight, correct_guess

def player(prev_play, opponent_history=[]):
    global My_history
    global counts, Dictionary_Of_Events, weights, guesses, guess_types, play_order

    actions = ['R', 'P', 'S']
    counter = {'R': 'P', 'P': 'S', 'S': 'R'}

    if not prev_play:
        weights, guesses, guess_types, counts = {}, {}, {}, {}
        Dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        My_history.clear()
        My_history = []
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        play_order = {"RR": 0, "RP": 0, "RS": 0,"PR": 0, "PP": 0,"PS": 0, "SR": 0,"SP": 0, "SS": 0}
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n) / 3
        predicted_optimal_guesses = {'R': 0, 'P': 0, 'S': 0}

        confidence_level_MH =  Total_sum_MH_confidence = 0
        strategy_detected = False
        highest_value = 0
        for i in range(1, 2):
            boost = min(1.0 + (i / (m + 1)), 1.3)

            guess_My_history, Probibility_Of_Events_MH, weight_MH, correct_guess_MH = n_gram(i, opponent_history, My_history, actions, 'Me')

            confidence_level_MH = entropy(Probibility_Of_Events_MH)

            confidence_level = confidence_level_MH
            Total_sum_MH_confidence += confidence_level
            Opponent_guessing_me = guess_My_history
            guess = None if Opponent_guessing_me is None else counter[Opponent_guessing_me]
            Type = 'Me'
            weighted_confidence_level = weight_MH * confidence_level * boost
            correct_guess = correct_guess_MH

            if guess and weighted_confidence_level != 0:
                predicted_optimal_guesses[guess] += math.log2(1 + max(0, weighted_confidence_level))
                guesses.setdefault(i, []).append(guess)
                guess_types.setdefault(i, []).append(Type)
                weights.setdefault(i, []).append(confidence_level)
                counts[i] = counts.get(i, 0) + correct_guess

            if counts.get(i, 0) / len(My_history) > 0.80:
                print(f"n={i}, accuracy={counts[i]/len(My_history):.2f}")
                optimal_n_gram = i
                predicted_guess = guess
                strategy_detected = True

        if strategy_detected and optimal_n_gram is not None:
            print(f"Detected n-gram strategy: n={optimal_n_gram}, type={guess_types[optimal_n_gram][-1]}")
            guess_type = guess_types[optimal_n_gram][-1]
            if guess_type == 'Me':
                counter_play = counter[counter[predicted_guess]]
            else:
                counter_play = counter[predicted_guess]
            My_history.append(counter_play)
            return counter_play

        if len(weights) == 0:
            counter_play = random.choice(actions)
            My_history.append(counter_play)
            return counter_play

        predicted_move = max(predicted_optimal_guesses, key=predicted_optimal_guesses.get)
        counter_play = counter[predicted_move]
    else:
        counter_play = random.choice(actions)

    # Debug: Compare Abbey prediction vs your model's prediction
    if len(My_history) > 3:
        most_common = max('RPS', key=lambda x: My_history.count(x))
        abbey_guess = {'R': 'P', 'P': 'S', 'S': 'R'}[most_common]
        ngram_guess, *_ = n_gram(1, opponent_history, My_history, ['R','P','S'], 'Me')
        print(f"[1-Gram Debug] Abbey would play: {abbey_guess}, Your 1-gram prediction: {ngram_guess}")

    My_history.append(counter_play)
    return counter_play


In [ ]:
#AI code
import math
import random
from collections import defaultdict

# Global variables
My_history = []
opponent_history = []
Dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
play_order = defaultdict(int)

def entropy(probabilities):
    entropy_val = 0
    for p in probabilities.values():
        if p > 0:
            entropy_val -= p * math.log2(p)
    max_entropy = math.log2(len(probabilities)) if probabilities else 0
    return 0 if max_entropy == 0 else 1 - (entropy_val / max_entropy)

def abbey(my_history):
    if len(my_history) < 2:
        return 'P'  # Counter to R

    last_two = ''.join(my_history[-2:])
    play_order[last_two] += 1

    potential = [my_history[-1] + move for move in 'RPS']
    sub_order = {k: play_order[k] for k in potential if k in play_order}
    prediction = max(sub_order, key=sub_order.get)[-1:] if sub_order else 'R'

    return {'R': 'P', 'P': 'S', 'S': 'R'}[prediction]

def n_gram(n, target_history, actions):
    counter = {'R': 'P', 'P': 'S', 'S': 'R'}
    correct_guess = 0

    if len(target_history) < n + 1:
        return None, {}, 0, 0

    pattern = tuple(target_history[-(n+1):-1])
    next_move = target_history[-1]
    Dictionary_Of_Events[pattern][next_move] += 1

    current_pattern = tuple(target_history[-n:])
    total = sum(Dictionary_Of_Events[current_pattern].values())

    if total == 0:
        return None, {}, 0, 0

    probabilities = {a: Dictionary_Of_Events[current_pattern].get(a, 0) / total for a in actions}
    guess = max(probabilities, key=probabilities.get)
    weight = (n / 5) * (total / (len(target_history) + 1))

    abbey_guess = abbey(My_history)
    if guess and counter[guess] == abbey_guess:
        correct_guess = 1

    return guess, probabilities, weight, correct_guess

def player(prev_play, opponent_history=[]):
    global My_history, Dictionary_Of_Events, play_order
    actions = ['R', 'P', 'S']
    counter = {'R': 'P', 'P': 'S', 'S': 'R'}

    if not prev_play:
        My_history.clear()
        opponent_history.clear()
        Dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        play_order = defaultdict(int)
        first_move = random.choice(actions)
        My_history.append(first_move)
        return first_move

    opponent_history.append(prev_play)

    if len(opponent_history) < 3:
        move = random.choice(actions)
        My_history.append(move)
        return move

    predicted_guesses = {a: 0 for a in actions}
    strategy_detected = False
    best_n = None
    best_guess = None
    total_guesses = 0

    for n in range(1, 2):  # Right now only 1-gram
        guess, probs, weight, correct = n_gram(n, My_history, actions)

        if guess:
            confidence = entropy(probs)
            adjusted_score = weight * confidence
            predicted_guesses[counter[guess]] += adjusted_score
            if correct:
                total_guesses += 1
            if total_guesses / len(My_history) > 0.8:
                strategy_detected = True
                best_n = n
                best_guess = guess

    if strategy_detected and best_guess:
        print(f"Detected opponent using {best_n}-gram model on you. Countering.")
        play = counter[counter[best_guess]]
    else:
        predicted_move = max(predicted_guesses, key=predicted_guesses.get)
        play = counter[predicted_move]

    # Debug: Compare Abbey to your model
    if len(My_history) > 3:
        my_ngram_guess, *_ = n_gram(1, My_history, actions)
        print(f"[Debug] Abbey would play: {abbey(My_history)}, 1-gram predicts me as: {my_ngram_guess}")

    My_history.append(play)
    return play

In [ ]:
#Bot against abbey
import copy
My_history = []
opponent_history = []
def entropy(Probibility_Of_Events):
          entropy = 0
          for probibility in Probibility_Of_Events:
              if Probibility_Of_Events[probibility] != 0:
                  #note - Why log2 used?
                  entropy = entropy - Probibility_Of_Events[probibility] * math.log2(Probibility_Of_Events[probibility])
          if len(Probibility_Of_Events) == 0 or not Probibility_Of_Events:
              return 0
          else:
            Max_entropy = math.log2(len(Probibility_Of_Events))
            if Max_entropy == 0:
              confidence_level = 0
            else:
              confidence_level = 1 - entropy/Max_entropy
          return confidence_level


def n_gram(n, opponent_history, My_history, actions, prev_play, repeat):
        pattern = tuple(My_history[-(n+1):-1])
        players_next_play = My_history[-1]
        correct_guess = 0
        counter = {'R':'P', 'P': 'S','S':'R'}
        if not repeat:
           Current_pattern = tuple(My_history[-n:])
           Previous_Dictionary_Of_Events = copy.deepcopy(Dictionary_Of_Events)
           Dictionary_Of_Events[pattern][players_next_play] += 1
        else:
           Current_pattern = tuple(My_history[-n:-1])

        for i in actions:
          if not repeat:
            if Dictionary_Of_Events[Current_pattern].get(i) == None:
                Dictionary_Of_Events[Current_pattern][i] = 0
          else:
            if Previous_Dictionary_Of_Events[Current_pattern].get(i) == None:
              Previous_Dictionary_Of_Events[Current_pattern][i] = 0

        if not repeat:
          total_sum = sum(Dictionary_Of_Events[Current_pattern].values())
        else:
          total_sum = sum(Previous_Dictionary_Of_Events[Current_pattern].values())

        Probibility_Of_Events = {}
        if total_sum == 0:
          weight = 0
          guess = None
        else:
          for i in actions:
              if not repeat:
                Probibility_Of_Events[i] = Dictionary_Of_Events[Current_pattern][i]/total_sum
              else:
                Probibility_Of_Events[i] = Previous_Dictionary_Of_Events[Current_pattern][i]/total_sum

          weight = (n/5)*(total_sum/(len(My_history)+1))

          guess = max(Probibility_Of_Events, key = Probibility_Of_Events.get)

        if not repeat:
          correct_guess = 0
          return guess, Probibility_Of_Events, weight, correct_guess
        else:
          if counter[guess] == prev_play:
            correct_guess += 1
          else:
            correct_guess = 0
          return guess, Probibility_Of_Events, weight, correct_guess


        #return dictionary_Of_events




def player(prev_play, opponent_history=[]):
    #note - reduce computation time by calculating the frequency of events and returning it back from previsous functions and only compute admendments to the
    #frequency dictionary once each round, based on the new influx of data.
    global My_history
    global Previous_Dictionary_Of_Events


    actions = ['R', 'P', 'S']
    counter = {'R':'P', 'P':'S','S':'R'}

    global counts
    global Dictionary_Of_Events
    global weights
    global guesses
    global guess_types

    if not prev_play:
        weights = {}
        guesses = {}
        guess_types = {}
        counts = {}
        Dictionary_Of_Events = defaultdict(lambda: defaultdict(int))
        My_history.clear()
        opponent_history.clear()
        play = random.choice(actions)
        My_history.append(play)
        return play

    opponent_history.append(prev_play)

    if len(opponent_history) > 2:
        Num_Of_games_played = len(opponent_history)
        n = Num_Of_games_played
        m = int(n)/3
        predicted_optimal_guesses = {'R':0, 'P':0, 'S':0}

        confidence_level_MH = 0
        confidence_level_PH = 0
        Total_sum_PH_confidence = 0
        Total_sum_MH_confidence = 0
        strategy_detected = False
        highest_value = 0
        for i in range(1,2):
          #note - boost seems quite arbitrary
            boost = 1.0 + (i / (m + 1))
            boost = min(boost, 1.3)

            guess_My_history, Probibility_Of_Events_MH, weight_MH ,correct_guess_M_1 = n_gram(i, opponent_history, My_history, actions, prev_play, False)
            guess_My_history_2, Probibility_Of_Events_MH_2, weight_MH_2, correct_guess= n_gram(i, opponent_history, My_history, actions, prev_play, True)


            confidence_level_MH = entropy(Probibility_Of_Events_MH)
            #note - compare weight to other n-gram weights see if there is an obvious n-gram strategy conducted by the opponent

            confidence_level = confidence_level_MH
            Total_sum_MH_confidence += confidence_level
            #Opponent guessing what my next move will be
            Opponent_guessing_me = guess_My_history
            if Opponent_guessing_me == None:
                guess = None
            else:
                # My prediction of what move the opponent will make
                guess = counter[Opponent_guessing_me]


            #note - Why is there 2 possible null cases? Find and fix
            if (guess):
                try: counts[i] += correct_guess
                except: counts[i] = correct_guess


            optimal_n_gram = None
            if counts[i]/len(My_history) > 0.80:
                  print(f"n={i}, accuracy={counts[i]/len(My_history):.2f}")
                  optimal_n_gram = i
                  predicted_guess = guess
                  strategy_detected = True



        if strategy_detected and optimal_n_gram is not None:
            print(f"Detected n-gram strategy: n={optimal_n_gram}, type={guess_types[optimal_n_gram][-1]}")
            #predicted_move = guesses[optimal_n_gram][-1]
            #guess_type = guess_types[optimal_n_gram][-1]
            counter_play = counter[counter[predicted_guess]]
            My_history.append(counter_play)
            return counter_play


        if len(weights) == 0:
          counter_play = random.choice(actions)
          My_history.append(counter_play)
          return counter_play


        predicted_move = random.choice(actions)
        counter_play = counter[predicted_move]


    else:
        counter_play = random.choice(actions)

    My_history.append(counter_play)
    #note - Focus on abbey and reducing computation time until we get the result needed
    #note - Research more about Markov chains to see if they would be a help

    return counter_play
